In [1]:
import numpy as np
import pandas as pd
import cassiopeia as cass
import datetime

from csv import writer


In [2]:
def add_match_history(puuid, games):
    global df, df2
    match_history = cass.MatchHistory(puuid=puuid,continent="AMERICAS", region='NA',begin_index=0,end_index=games,queue=cass.Queue.ranked_solo_fives)
    
    for match in match_history:
        try:
            stat_import = pd.DataFrame(columns=df.columns,index=[0])
        except:
            stat_import = pd.DataFrame(index=[0])
            
        try:
            if str(match.id) in df["ID"].values or str(match.id)[4:] in df["ID"].values:
                continue
        except:
            pass
        
        if match.duration < datetime.timedelta(hours=0, minutes=15, seconds=0):
            continue
        
        player_list = []
        # Making the player objects and adding to list above
        for i in range(0,10):
            # pi = participant i
            exec('p'+str(i)+'='+'match.participants['+str(i)+']')
            # add pi to list
            exec('player_list.append(p'+str(i)+')')
        
        if type(match.id) == str:
            stat_import['ID'] = match.id[4:]
        else:
            stat_import['ID'] = str(match.id)
        stat_import['B_Win'] = match.blue_team.win

        try:
             
            for player in player_list:
                
                if player.side.value == 100:
                    if player.individual_position.name == 'top_lane':

                        stat_import['B_Top'] = player.champion.name
                        stat_import['B_Top_Kills'] = player.stats.kills
                        stat_import['B_Top_KDA'] = player.stats.kda
                        
                    elif player.individual_position.name == 'jungle':

                        stat_import['B_Jg'] = player.champion.name
                        stat_import['B_Jg_Kills'] = player.stats.kills
                        stat_import['B_Jg_KDA'] = player.stats.kda

                    elif player.individual_position.name == 'mid_lane':

                        stat_import['B_Mid'] = player.champion.name
                        stat_import['B_Mid_Kills'] = player.stats.kills
                        stat_import['B_Mid_KDA'] = player.stats.kda

                    elif player.individual_position.name == 'bot_lane':

                        stat_import['B_Bot'] = player.champion.name
                        stat_import['B_Bot_Kills'] = player.stats.kills
                        stat_import['B_Bot_KDA'] = player.stats.kda

                    elif player.individual_position.name == 'utility':

                        stat_import['B_Sup'] = player.champion.name
                        stat_import['B_Sup_Kills'] = player.stats.kills
                        stat_import['B_Sup_KDA'] = player.stats.kda

                if player.side.value == 200:
                    if player.individual_position.name == 'top_lane':

                        stat_import['R_Top'] = player.champion.name
                        stat_import['R_Top_Kills'] = player.stats.kills
                        stat_import['R_Top_KDA'] = player.stats.kda

                    elif player.individual_position.name == 'jungle':

                        stat_import['R_Jg'] = player.champion.name
                        stat_import['R_Jg_Kills'] = player.stats.kills
                        stat_import['R_Jg_KDA'] = player.stats.kda

                    elif player.individual_position.name == 'mid_lane':

                        stat_import['R_Mid'] = player.champion.name
                        stat_import['R_Mid_Kills'] = player.stats.kills
                        stat_import['R_Mid_KDA'] = player.stats.kda

                    elif player.individual_position.name == 'bot_lane':

                        stat_import['R_Bot'] = player.champion.name
                        stat_import['R_Bot_Kills'] = player.stats.kills
                        stat_import['R_Bot_KDA'] = player.stats.kda

                    elif player.individual_position.name == 'utility':

                        stat_import['R_Sup'] = player.champion.name
                        stat_import['R_Sup_Kills'] = player.stats.kills
                        stat_import['R_Sup_KDA'] = player.stats.kda
                        
            # stat_import.to_csv(path_or_buf="matches2.csv",mode='a',index=False,header=False)
                    
            df = pd.concat([df,stat_import], ignore_index=True)
            df2 = pd.concat([df2,stat_import], ignore_index=True)
                
        except:
           pass

    return find_next_player(match_history)
    

In [3]:
# Function that selects a random player from the last player's previous games
def find_next_player(history):
    game = 0
    New_Player_Attempts = 0
    while True:
        New_Player_Attempts += 1
        # the match to select a new player from
        match = history[game]
        # selecting random number to get the random player from match
        randNum = np.random.randint(11)
        player = match.participants[randNum]
        # If player has already been checked
        if player.summoner.name in already_checked:
            # only 10 players in a match, if all players been checked go to next game
            if New_Player_Attempts == 10:
                game += 1
                New_Player_Attempts = 0
                continue
            else:
                continue
        break
    print(player.summoner.name)
    # keep track of the account that has been checked
    already_checked.append(player.summoner.name)
    # Returns the puuid of the new player
    return player.summoner.puuid

In [4]:
cass.set_riot_api_key("RGAPI-facd70cc-4cd9-432f-92ee-6c4210435e51")
player = cass.Summoner(name="Artezzaa", region="NA")


In [5]:
i = 1
# list so we don't check the same player's games more than once
already_checked = [player.name]
# DataFrame that keeps track of all the match data over whole process
df = pd.DataFrame()
# Get the match info for the seed player, with this number of matches
next_puuid = add_match_history(player.puuid, 50)
# DataFrame to output blocks of matches at once
df2 = pd.DataFrame(df)
# Print out the first block of matches from seed account with column names
df2.to_csv("matches3.csv",index=False,mode='w')
# Number of players you want to get matches from
while i != 100:
    # empty output dataframe to stop duplicates in file
    df2 = df2.iloc[0:0]
    # Get the match info for the randomly selected player, with this number of matches
    next_puuid = add_match_history(next_puuid, 50)
    # Print out the next block of matches from random selected account without columns
    df2.to_csv("matches3.csv",index=False,mode='a',header=False)
    i += 1

Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/Artezzaa
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/c_dgoK4Dv3ZHJjkyvD1i8Wv_py3AcLgY_x7trjSgWeyms-aV1AtEONdMKJ1eP79ZAvFSu5Dn3_elsA/ids?start=0&count=50&queue=420
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4254552847
Making call: https://cdn.merakianalytics.com/riot/lol/resources/patches.json
Making call: https://ddragon.leagueoflegends.com/api/versions.json
Making call: https://ddragon.leagueoflegends.com/cdn/12.5.1/data/en_US/championFull.json
Making call: https://ddragon.leagueoflegends.com/realms/na.json
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4251572080
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4251403578
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4250631950
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4249706191
Making ca

APIError: The Riot API experienced an internal error on the request. You may want to retry the request after a short wait or continue without the result. The received error was 503: "Service Unavailable"

In [ ]:
df.to_csv('matches_dump.csv', index=False, mode='w')

,ID,B_Win,R_Win,B_Top,B_Top_Kills,B_Top_KDA,B_Jg,B_Jg_Kills,B_Jg_KDA,B_Mid,...,R_Jg_KDA,R_Mid,R_Mid_Kills,R_Mid_KDA,R_Bot,R_Bot_Kills,R_Bot_KDA,R_Sup,R_Sup_Kills,R_Sup_KDA
0,4254552847,False,True,Jayce,5,1.2,Ekko,3,1.230769,Veigar,...,2.875000,Gragas,13,1.857143,Jhin,5,2.400000,Lux,4,2.3
1,4251572080,True,False,Aatrox,3,1.5,Jarvan IV,10,21.000000,Akali,...,1.000000,Karma,3,1.666667,Ezreal,6,0.875000,Leona,1,1.0
2,4251403578,False,True,Camille,7,1.375,Master Yi,3,1.142857,Yasuo,...,8.000000,Katarina,14,10.5,Kai'Sa,7,8.500000,Leona,1,10.0
3,4250631950,False,True,Akali,9,2.5,Kayn,5,1.545455,Cassiopeia,...,2.333333,Yasuo,17,2.888889,Kai'Sa,12,4.166667,Yuumi,1,5.2
4,4249706191,True,False,Singed,4,14.0,Talon,13,4.000000,Ahri,...,4.666667,Lissandra,1,1.5,Jhin,4,0.857143,Nautilus,0,0.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,4242691695,True,False,Yorick,0,0.0,Nasus,7,4.666667,Anivia,...,3.500000,Soraka,4,6.0,Jinx,16,5.500000,Zilean,3,5.5
228,4242618082,False,True,Tryndamere,3,1.125,Zac,8,4.750000,Veigar,...,5.250000,Nocturne,10,23.0,Vayne,3,0.461538,Zilean,3,4.0
229,4242534626,False,True,Warwick,2,0.571429,Graves,1,0.400000,Ahri,...,5.000000,LeBlanc,8,9.0,Caitlyn,4,10.000000,Janna,2,12.0
230,4242518721,True,False,Fiora,7,3.0,Gwen,13,6.000000,Irelia,...,1.625000,Mordekaiser,8,2.0,Jhin,3,1.375000,Zilean,2,2.0


In [ ]:
# https://github.com/meraki-analytics/cassiopeia/tree/master/examples